In [1]:
import requests
import json
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import plotly.express as px


# Fetching GeoJSON data
url = "https://maps.amsterdam.nl/open_geodata/geojson_lnglat.php?KAARTLAAG=INDELING_BUURT&THEMA=gebiedsindeling"

response = requests.get(url)
data = response.json()

# Load the GeoJSON data into a GeoDataFrame and set the CRS to EPSG:4326
gdf = gpd.GeoDataFrame.from_features(data, crs="EPSG:4326")

# Define the API endpoint and API key
api_key = "tVIbkYr2rievbsl6R48uBl5qzuQeeYpP"
base_url = "https://api.deelfietsdashboard.nl/dashboard-api/parkeertelling?"
headers = {"Content-type": "application/json"}

# Create an empty DataFrame to store all the responses
combined_df = pd.DataFrame()

# Iterate through the GeoDataFrame and make API requests for each polygon
for index, row in gdf.iterrows():
    coordinates = row['geometry'].exterior.coords[:-1]  # Get the coordinates of the polygon's exterior ring
    polygon_coordinates = [[coord[0], coord[1]] for coord in coordinates]
    
    data = {
        "timestamp": "2023-11-01T14:00:00Z",
        "geojson": {
            "type": "Polygon",
            "coordinates": [polygon_coordinates]
        }
    }

    apiurl = f"{base_url}apikey={api_key}"

    response = requests.post(apiurl, data=json.dumps(data), headers=headers)

    if response.status_code == 200:
        json_string = response.text
        data = json.loads(json_string)
        buurt_value = gdf.at[index, 'Buurt']
        print(f"Collecting Data for Polygon {index} ({buurt_value})\n")
        df = pd.DataFrame(data)
        df['Polygon'] = index  # Add a column to identify the polygon
        combined_df = pd.concat([combined_df, df], ignore_index=True)
    else:
        print(f"Error: Unable to get data for Polygon {index}")
        print(response.text)

# Save the combined data to a CSV file
combined_df.to_csv("combined_data.csv", index=False)

# Display the combined DataFrame
print("Combined Data:\n")
print(combined_df)



AttributeError: module 'numpy' has no attribute 'float'.
`np.float` was a deprecated alias for the builtin `float`. To avoid this error in existing code, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [ ]:
# Initialize an empty dictionary to store the aggregated data
aggregated_data = {
    'Buurt': [],
    'bicycle': [],
    'cargo_bicycle': [],
    'moped': [],
    'scooter': []
}

current_polygon = None
counts = {
    'bicycle': 0,
    'cargo_bicycle': 0,
    'moped': 0,
    'scooter': 0
}

for index, row in combined_df.iterrows():
    polygon = row['Polygon']
    if polygon != current_polygon:
        if current_polygon is not None:
            buurt_value = gdf.at[current_polygon, 'Buurt']
            aggregated_data['Buurt'].append(buurt_value)
            aggregated_data['bicycle'].append(counts['bicycle'])
            aggregated_data['cargo_bicycle'].append(counts['cargo_bicycle'])
            aggregated_data['moped'].append(counts['moped'])
            aggregated_data['scooter'].append(counts['scooter'])
        
        current_polygon = polygon
        counts = {
            'bicycle': 0,
            'cargo_bicycle': 0,
            'moped': 0,
            'scooter': 0
        }
    
    vehicle_type = row[0]
    count = row[1]
    
    if vehicle_type in counts:
        counts[vehicle_type] += count

# Append the last group of counts
if current_polygon is not None:
    buurt_value = gdf.at[current_polygon, 'Buurt']
    aggregated_data['Buurt'].append(buurt_value)
    aggregated_data['bicycle'].append(counts['bicycle'])
    aggregated_data['cargo_bicycle'].append(counts['cargo_bicycle'])
    aggregated_data['moped'].append(counts['moped'])
    aggregated_data['scooter'].append(counts['scooter'])

# Create a new DataFrame from the aggregated data
aggregated_df = pd.DataFrame(aggregated_data)

# Display the aggregated DataFrame
print("Aggregated Data:\n")
print(aggregated_df)
aggregated_df.to_csv("aggregated_data.csv", index=False)

gdf = gpd.read_file(url)

# Define the columns to include in the heatmap
columns_to_include = ['bicycle', 'cargo_bicycle', 'moped', 'scooter']

# Create a new column in the aggregated_df to represent the total counts
aggregated_df['Total'] = aggregated_df[columns_to_include].sum(axis=1)

# Merge the GeoDataFrame with the aggregated data
merged = gdf.merge(aggregated_df, on='Buurt', how='left')

print(merged)

merged.set_index('Buurt', inplace=True)

# Create an interactive heatmap using Plotly Express
fig = px.choropleth_mapbox(
    merged,
    geojson=merged.geometry,
    locations=merged.index,  # Use the index as locations
    color='Total',
    color_continuous_scale='YlOrRd',
    range_color=(0, merged['Total'].max()),  # Adjust the color scale range
    title="Interactive Heatmap of Total Counts by Buurt",
    mapbox_style="carto-positron",
    zoom=10, center = {"lat": 52.3676, "lon": 4.9041},
)

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()
